In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import json

# Path to the JSON file
file_path = '../input/yelp-dataset/yelp_academic_dataset_business.json'
?
data = []

# Open and read the JSON file line by line
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Create a DataFrame from the list of JSON objects
business = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
business.head()

#business_id : Primary key

In [ ]:
print("Number of Different cities:", len(business["city"].unique()))
print("Number of Different states:", len(business["state"].unique()))
print("Range of latitude: " ,business["latitude"].min(), business["latitude"].max())
print("Range of longitude: " ,business["longitude"].min(), business["longitude"].max())
print("Range of stars: ", business["stars"].min(), business["stars"].max())
print("Range of the number of reviews: ", business["review_count"].min(), business["review_count"].max())
print("is_open: ", business["is_open"].unique())

In [ ]:
#Business Categories
def str_list(string):
  if pd.isna(string):
    return []
  else:
    return string.split(", ")

cat = business["categories"].apply(str_list)
cat_list = []
for i in range(len(cat)):
  if len(cat[i]) == 0:
    cat_list = cat_list
  else:
    for j in range(len(cat[i])):
      cat_list.append(cat[i][j])

In [ ]:
pd.Series(cat_list).value_counts()

EDA for Business Table: 

business_id is the Primary Key here (Unique value)


name indiacates the Business name


address: Business Address


city: City (DISTINCT: 1416)


state: State (DISTINCT: 27)


postal_code: Zip code (Character)


latitude: Latitude of the location Range of latitude:  27.555127 53.6791969



longitude: Longitude of the location Range of longitude:  -120.095137 -73.2004570502


stars: Rating [1.0, 5.0]


review_count: # of reviews [5, 7568]


is_open: Whether the business is operating or not (0 or 1)


attributes: 39 attributes - Need to create a new table with every attribute as a new column


categories: List of business categories in string (DISTINCT: 1311) : create to make a new table


hours: Operation hours by day in json format - Create new table

In [ ]:
att = list(enumerate(business["attributes"]))
atts = []
for i in range(len(att)):
  atts.append(att[i][1])
att_df = pd.json_normalize(atts)
att_df.shape

In [ ]:
att_df = pd.concat([business[["business_id"]], att_df], axis = 1)
att_df["ID"] = range(len(att_df))

In [ ]:
att_df.head(5)

In [ ]:
#category_df
cat_df = business[["business_id", "categories"]]
category_df = (cat_df.assign(categories=cat_df['categories'].str.split(','))
       .explode('categories', ignore_index=True))
category_df.head(10)

In [ ]:
category_df["ID"] = range(len(category_df))
category_df.head(10)

In [ ]:
#Create hour table
hour = list(enumerate(business["hours"]))
hour_list = []
for i in range(len(hour)):
  hour_list.append(hour[i][1])
hour_df = pd.json_normalize(hour_list)
hour_df.head(10)

In [ ]:
hours_df = pd.concat([business[["business_id"]], hour_df], axis = 1)
hours_df["ID"] = range(len(hours_df))

In [ ]:
business = business.drop(["attributes", "categories", "hours"], axis = 1)

In [ ]:
business.head(5)


In [ ]:
import pandas as pd
import json

# Path to the JSON file
file_path = '../input/yelp-dataset/yelp_academic_dataset_user.json'

data = []

# Open and read the JSON file line by line
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Create a DataFrame from the list of JSON objects
user = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
user.head()

In [ ]:
print("Range of review count", min(user["review_count"]), max(user["review_count"]))
print("Range of yelping_since", min(user["yelping_since"]), max(user["yelping_since"]))
print("Range of useful", min(user["useful"]), max(user["useful"]))
print("Range of funny", min(user["funny"]), max(user["funny"]))
print("Range of cool", min(user["cool"]), max(user["cool"]))
print("Range of fans", min(user["fans"]), max(user["fans"]))

EDA for User Table

user_id: User ID (Primary Key)


name: Name of the user


review_count: Number of reviews by users [0, 17473]


yelping_since: Date of sign up ("YYYY-MM-DD HH:MM:SS") [2004-10-14 23:50:06, 2020-02-16 18:44:01]


useful: [0, 206296] Range


funny: [0, 185823] Range


cool: [0, 195814] Range


elite: List of years 


friends: List of friends (Recommended) 


fans: [0, 12497] 


Additional set of columns with categorized columns for compliment types

In [ ]:
#Review

# Path to the file
file_path = '../input/yelp-dataset/yelp_academic_dataset_review.json'

data = []
max_rows = 1000000  # Limit to one million rows

with open(file_path, 'r') as file:
    for i, line in enumerate(file):
        if i >= max_rows:  # Break the loop after reading one million lines
            break
        data.append(json.loads(line))

# Create a DataFrame from the list of JSON objects
review = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
review.head()

#reivew_id: Primary key 

EDA for review table 

review_id is the primary key here


user_id Indicates the user 


business_id indicates the business for which review was written


further we have stars which indicate the number of stars given by the user 


Review categorization into useful, funny and/or cool: Helps to identify the review provided by the user. (0 and 1 used as indicator, 1 indicates the particular review belongs to that category or not)

Text indicates the review written by the user

In [ ]:
import pandas as pd
import json

# Path to the JSON file
file_path = '../input/yelp-dataset/yelp_academic_dataset_tip.json'

data = []

# Open and read the JSON file line by line
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Create a DataFrame from the list of JSON objects
tip = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
tip.head()

In [ ]:
#AGNUgVwnZUey3gcPCJ76iw

duplicate_values = tip['user_id'].duplicated()
duplicate_values.value_counts()

EDA for Tip Table

user_id : can be multiple


business_id: Business ID (There can be same business ids)



text: Text about tips


date: Dates of tips [2009-04-16 13:11:49, 2022-01-19 20:38:55]


compliment_count: [0, 6]

In [ ]:
import pandas as pd
import json

# Path to the JSON file
file_path = '../input/yelp-dataset/yelp_academic_dataset_checkin.json'

data = []

# Open and read the JSON file line by line
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Create a DataFrame from the list of JSON objects
checkin = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
checkin.head()

EDA for checkin :


business_id: Business ID


date: list of checkin dates ("YYYY-MM-DD HH:MM:SS")

In [ ]:
!pip install pymysql
import pymysql
import sqlalchemy

from sqlalchemy import create_engine

In [ ]:
username = 'team4'  # replace the hash symbol with your team number
password = 'jmpn4'
host = 'testproject.mysql.database.azure.com'  # do not change this
database = 'team4'  # replace the hash symbol with your team number

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"

ssl_args = {
    'ssl_ca': 'path_to/ca-cert.pem',
    'ssl_cert': 'path_to/client-cert.pem',
    'ssl_key': 'path_to/client-key.pem'
}

engine = create_engine(
    connection_string,
    connect_args={'ssl': ssl_args}
)

In [ ]:
business.to_sql(name = "business", con = engine, if_exists='replace', index=True, chunksize=10000)

In [ ]:
att_df.to_sql(name = "attributes", con = engine, if_exists='replace', index=True, chunksize=10000)

In [ ]:
category_df.to_sql(name = "categories",  con = engine, if_exists='replace', index=True, chunksize=10000)

In [ ]:
hours_df.to_sql(name = "hours", con = engine, if_exists='replace', index=True, chunksize=10000)

In [ ]:
review.to_sql(name = "review", con = engine, if_exists='replace', index=True, chunksize=50000) 

In [ ]:
tip.to_sql(name = "tip", con = engine, if_exists='replace', index=True, chunksize=50000) 

In [ ]:

checkin_modified['date'] = checkin['date'].apply(lambda x: x[:10] if isinstance(x, str) else x)

# Now, write the modified DataFrame to SQL, excluding the dropped row
checkin_modified.to_sql(name='checkin', con=engine, if_exists='replace', index=False, chunksize=50000)

In [ ]:
user_modified = user.copy()
user_modified['friends'] = user['friends'].apply(lambda x: x[:10] if isinstance(x, str) else x)

# Now, write the modified DataFrame to SQL, excluding the dropped row
user_modified.to_sql(name='user', con=engine, if_exists='replace', index=False, chunksize=50000)